In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import random

from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

pal = sns.color_palette("pastel").as_hex()

random.seed(10)

%matplotlib inline

In [7]:
import warnings
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")

In [2]:
# importing the training data
df = pd.read_csv('data/train.csv')

In [3]:
df.shape

(1460, 81)

In [4]:
df.loc[1699,'Garage Yr Blt'] = 2007
df.loc[1712,'Garage Type'] = np.NaN
df.loc[1712,'Garage Cars'] = 0
df.loc[1712,'Garage Area'] = 0

In [8]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Garage Yr Blt,Garage Type,Garage Cars,Garage Area
0,1.00,60.00,RL,65.00,8450.00,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.00,5.00,2003.00,2003.00,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.00,Gd,TA,PConc,Gd,TA,No,GLQ,706.00,Unf,0.00,150.00,856.00,GasA,Ex,Y,SBrkr,856.00,854.00,0.00,1710.00,1.00,0.00,2.00,1.00,3.00,1.00,Gd,8.00,Typ,0.00,NaN,Attchd,2003.00,RFn,2.00,548.00,TA,TA,Y,0.00,61.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,0.00,2.00,2008.00,WD,Normal,208500.00,nan,nan,nan,nan
1,2.00,20.00,RL,80.00,9600.00,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6.00,8.00,1976.00,1976.00,Gable,CompShg,MetalSd,MetalSd,None,0.00,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.00,Unf,0.00,284.00,1262.00,GasA,Ex,Y,SBrkr,1262.00,0.00,0.00,1262.00,0.00,1.00,2.00,0.00,3.00,1.00,TA,6.00,Typ,1.00,TA,Attchd,1976.00,RFn,2.00,460.00,TA,TA,Y,298.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,0.00,5.00,2007.00,WD,Normal,181500.00,nan,nan,nan,nan
2,3.00,60.00,RL,68.00,11250.00,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.00,5.00,2001.00,2002.00,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.00,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.00,Unf,0.00,434.00,920.00,GasA,Ex,Y,SBrkr,920.00,866.00,0.00,1786.00,1.00,0.00,2.00,1.00,3.00,1.00,Gd,6.00,Typ,1.00,TA,Attchd,2001.00,RFn,2.00,608.00,TA,TA,Y,0.00,42.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,0.00,9.00,2008.00,WD,Normal,223500.00,nan,nan,nan,nan
3,4.00,70.00,RL,60.00,9550.00,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7.00,5.00,1915.00,1970.00,Gable,CompShg,Wd Sdng,Wd Shng,None,0.00,TA,TA,BrkTil,TA,Gd,No,ALQ,216.00,Unf,0.00,540.00,756.00,GasA,Gd,Y,SBrkr,961.00,756.00,0.00,1717.00,1.00,0.00,1.00,0.00,3.00,1.00,Gd,7.00,Typ,1.00,Gd,Detchd,1998.00,Unf,3.00,642.00,TA,TA,Y,0.00,35.00,272.00,0.00,0.00,0.00,NaN,NaN,NaN,0.00,2.00,2006.00,WD,Abnorml,140000.00,nan,nan,nan,nan
4,5.00,60.00,RL,84.00,14260.00,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8.00,5.00,2000.00,2000.00,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.00,Gd,TA,PConc,Gd,TA,Av,GLQ,655.00,Unf,0.00,490.00,1145.00,GasA,Ex,Y,SBrkr,1145.00,1053.00,0.00,2198.00,1.00,0.00,2.00,1.00,4.00,1.00,Gd,9.00,Typ,1.00,TA,Attchd,2000.00,RFn,3.00,836.00,TA,TA,Y,192.00,84.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,0.00,12.00,2008.00,WD,Normal,250000.00,nan,nan,nan,nan


In [10]:
df.drop(['Id'], axis=1, inplace=True)

In [31]:
# features to be converted
to_cat = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual','Functional',
          'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC', 'CentralAir', 'GarageFinish']

# keys for conversions
cat_key = {'five':
           {'Gd':3, 'TA':2, 'Ex':4, 'Po': 0, 'Fa':1},
           'five_NA':
           {'Gd':4, 'TA':3, 'Ex':5, 'Po': 1, 'Fa':2, 'N':0, np.NaN:0},
           'function':
           {'Typ':7, 'Mod':4, 'Min2':5, 'Maj1':3, 'Min1':6, 'Sev':1, 'Sal':0, 'Maj2':2, np.NaN:0},
           'binary':
           {'N':0, 'Y':1, np.NaN:0},
           'finish':
           {'RFn':2, 'Unf':1, 'Fin':3, np.NaN:0}}

# assigning each column a key dict
num_key = {'ExterQual':'five_NA', 'ExterCond':'five_NA', 'BsmtQual':'five_NA',
           'BsmtCond':'five_NA', 'HeatingQC':'five_NA', 'KitchenQual':'five_NA',
           'Functional':'function', 'FireplaceQu':'five_NA', 'GarageQual':'five_NA',
           'GarageCond':'five_NA', 'PoolQC':'five_NA', 'CentralAir':'binary',
           'GarageFinish':'finish'}

# converting
for c in to_cat:
    p = str(f'{c}_num')
    df[p] = df[c].map(lambda x: cat_key[num_key[c]][x])

In [34]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [35]:
# features to be converted
to_age = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']

# new feature names
age_names = ['House Age', 'Remod/Add Age', 'Garage Age', 'Years Since Sale']

# dictionary of old names and new names
ager = dict(zip(to_age,age_names))

# convert by subtracting from current year
for n, v in ager.items():
    df[v] = df[n].map(lambda x: 2018-x)

In [39]:
df.drop([c for c in to_age + to_cat], axis=1, inplace=True)

In [42]:
nulls = {'col':[], 'null':[]}
for c in df.columns:
    if df[c].isnull().sum() > 0:
        nulls['col'].append(c)
        nulls['null'].append(df[c].isnull().sum())
        
pd.DataFrame(nulls).sort_values('null', ascending=False)

,col,null
64,Garage Type,1462
66,Garage Area,1461
63,Garage Yr Blt,1461
65,Garage Cars,1461
57,MiscFeature,1408
...,...,...
34,1stFlrSF,2
1,MSZoning,2
36,LowQualFinSF,2
37,GrLivArea,2
